In [ ]:
Use either resource, client object to perform the following tasks. 

1 - Create 3 snapshot in your aws account manually 
b - write a function to list: 
        snapshot id
        sizes :
        creation time, 
        
3 - use csv module to write the the infomation of these snapshot as an inventory file. 

In [84]:
import boto3 
from pprint import pprint 
import csv 

from botocore.exceptions import ClientError 

resource = boto3.session.Session().resource('ec2', 'us-east-1')
client =  boto3.session.Session().client('ec2', 'us-east-1')
sts = boto3.session.Session().client('sts')


def account_id():
    return sts.get_caller_identity()['Account']
 
    
def get_snapshot():
    snap_account = account_id()
    response = resource.snapshots.filter(OwnerIds=[snap_account])
    ## Using csv to dump snaps info 
    csv_header = ["Snap-id", "snp-Size", "Start-Time"] 
    with open("snapshots_inventory.csv", "w", newline="") as file:
        csv_w = csv.writer(file)
        csv_w.writerow(csv_header)
        print("Writing snapshot info to an inventory file.")
        for each_snap in response:
            csv_w.writerow([each_snap.id, each_snap.volume_size, each_snap.start_time.strftime("%Y-%m-%d")])
        print("Finish!")        
        
    
def get_volumes():
    response =  resource.volumes.all()
    for each_volume in response:
        if each_volume.tags:
            for each_tags in each_volume.tags:
                print(f"This volume was taged: {each_volume.id}, Tags: {each_tags.get('Value')}")
        else:
            ## GET THE UNTAGED VOLUMES
            ## DELETE UNTAGED VOLUMES
            print("deleting untaged volumes")

    ## How do we get the volumes that are not-inuse
    volumes_inused = [{'Name': 'status', 'Values': ['in-use']}]
    response = resource.volumes.filter(Filters=volumes_inused)
    for each_volume in response:
        print(each_volume.id)
        
        
def delete_unused_untaged_volumes():
    ## First let's get the volume state 
    for each_volume in resource.volumes.all():
        if each_volume.state != 'in-use' and not each_volume.tags:
            print("Neither taged nor in-used\n So deleting the volume")
            print(each_volume.delete())
        
    
if __name__ == "__main__":
#     get_volumes()
#     get_snapshot()
    delete_unused_untaged_volumes()

    

In [ ]:


## we want to delete only volumes that are neither taged nor in-used
